# **PLOT VERTICAL PROFILES SEVERAL EXPERIMENTS VERSUS OBSERVATIONS**

In [1]:
import metview as mv
import os
from datetime import date, datetime, timedelta
os.environ["METVIEW_PYTHON_DEBUG"] = "1"
os.environ["MIR_GRIB_INPUT_BUFFER_SIZE"] = "7688961960"
os.environ["MARS_READANY_BUFFER_SIZE"] = "7688961960"
os.environ["MIR_CACHE_PATH"] = "${SCRATCH}"
os.environ["TMPDIR"] = "${SCRATCHDIR}"

## Define your variables

In [3]:
profile_type = "skewt"                  # you can choose: "tephigram", "skewt" or "emagram" 
use_mars = False                         # use_mars = True if it is the first time you run that day/time. It will retrieve the data
cape_instability = True                 # cape_instability = True to plot instability indices inside the plot
cape_data = "prof_fc2"                  # Change the database that you want to use for the calculation of the instability indices: prof_fc1 or prof_fc2 

fdate = datetime(2023,12,3)             # forecast initialisation day
step = 12                               # time step of the forecast 
area = [56.54,5.65,72.97,36.16]         # area where you want to retrieve the data 
expver1 = 1                             # expver of the first model. expver=1 is assigned to"class": "od", change the class to "rd" when needed
expver2 = "i4ql"                        # expver of the second model

# Choose the location you want to plot. 
#location = [45.53,8.67]   # flat area low with radiosounding obs
#location = [46.68,12.257] # temp diff: -8.5C, orog diff 2559: 327 m
#location = [46.076,12.769] # LOW temp diff: 10.8C, orog diff 2559: -17 m
#location = [46.5144,10.317] # LOW temp diff: 5C, orog diff 2559 and tco1279: 2200 m
#location = [44.832,11.621] # LOW temp diff: -1.3C, orog diff 2559: -3 m
#location = [45.927,10.291] # LOW temp diff: 1.55, orog diff 2559: 981 m
location = [65.4,25.6] # Location valley Birgit plot


In [4]:
adate = fdate.strftime("%Y%m%d")
print(adate)
valid_date = fdate + timedelta(hours=step)
valid_time = mv.hour(valid_date)
vtime=int("%2i" % valid_time)
print(valid_date)
print(valid_time)
if expver1 == 1:
    strexpver1 = "oper"
else:
    strexpver1 = expver1
    
# Definition file names    
filename_fc1 = "fc_prof_" + strexpver1 + "_" + fdate.strftime("%Y%m%d") + "_" + str("%02d" % step) + ".grib"
filename_fc2 = "fc_prof_" + expver2 + "_" + fdate.strftime("%Y%m%d") + "_" + str("%02d" % step) + ".grib"
skt_fc1 = "fc_skt_" + strexpver1 + "_" + fdate.strftime("%Y%m%d") + "_" + str("%02d" % step) + ".grib" 
skt_fc2 = "fc_skt_" + expver2 + "_" + fdate.strftime("%Y%m%d") + "_" + str("%02d" % step) + ".grib" 
t2_fc1 = "fc_2t_" + strexpver1 + "_" + fdate.strftime("%Y%m%d") + "_" + str("%02d" % step) + ".grib" 
t2_fc2 = "fc_2t_" + expver2 + "_" + fdate.strftime("%Y%m%d") + "_" + str("%02d" % step) + ".grib" 
sp_fc1 = "fc_sp_" + strexpver1 + "_" + fdate.strftime("%Y%m%d") + "_" + str("%02d" % step) + ".grib" 
sp_fc2 = "fc_sp_" + expver2 + "_" + fdate.strftime("%Y%m%d") + "_" + str("%02d" % step) + ".grib" 

print(filename_fc1)
print(filename_fc2)

20231203
2023-12-03 12:00:00
12.0
fc_prof_oper_20231203_12.grib
fc_prof_i4ql_20231203_12.grib


## Utility function to generate text for plot title

In [5]:

def build_title_text(prof_fc):
    # fc or an
    info = mv.thermo_data_info(prof_fc)
    t2 = "Run: {} UTC +{}h Valid: {} UTC".format(
    fdate.strftime("%Y-%m-%d %H"),
    str("%02d" % step),
    valid_date.strftime("%Y-%m-%d %H")
    )

    return [t2]

## Getting data


In [6]:
# getting forecast data from MARS
if use_mars:
    ret_core1 = {
        "date": adate,
        "time": 0,
        "step": step,
        "type": "fc",
        "class": "od",
        "levtype": "ml",
        "expver": expver1,
        "area": area,
        "grid": [0.1, 0.1],
    }

    # forecast fields on model levels 1-137 (bottom is ML=137)
    fs_ml1 = mv.retrieve(
        **ret_core1,
        levelist=[1, "TO", 137],
        param=["t", "q", "u", "v","lnsp"],
    )

    mv.write(filename_fc1,fs_ml1)
    
    ret_core2 = {
         "date": adate,
         "time": 0,
         "step": step,
         "type": "fc",
         "class": "rd",
         "levtype": "ml",
         "expver": expver2,
         "area": area,
         "grid": [0.05, 0.05],
     }

     # forecast fields on model levels 60-137 (bottom is ML=137)
    fs_ml2 = mv.retrieve(
        **ret_core2,
        levelist=[1, "TO", 137],
        param=["t", "q", "u", "v", "lnsp"],
    )

    mv.write(filename_fc2,fs_ml2)
            
# FC
g1 = mv.read(filename_fc1)
g2 = mv.read(filename_fc2)


In [7]:
if use_mars:
    skt_area1 = mv.retrieve(
        date= adate,
        time= 0,
        step= step,
        type= "fc",
        class_= "od",
        levtype= "sfc",
        expver= expver1,
        param="skt",
        area= area,
        grid= [0.1, 0.1]
    )

    t2_area1 = mv.retrieve(
        date= adate,
        time= 0,
        step= step,
        type= "fc",
        class_= "od",
        levtype= "sfc",
        expver= expver1,
        param="2t",
        area= area,
        grid= [0.1, 0.1]
    )

    sp_area1 = mv.retrieve(
        date= adate,
        time= 0,
        step= step,
        type= "fc",
        class_= "od",
        levtype= "sfc",
        expver= expver1,
        param="sp",
        area= area,
        grid= [0.1, 0.1]
    )

    skt_area2 = mv.retrieve(
        date= adate,
        time= 0,
        step= step,
        type= "fc",
        class_= "rd",
        levtype= "sfc",
        expver= expver2,
        param="skt",
        area= area,
        grid= [0.1, 0.1]
    )

    t2_area2 = mv.retrieve(
        date= adate,
        time= 0,
        step= step,
        type= "fc",
        class_= "rd",
        levtype= "sfc",
        expver= expver2,
        param="2t",
        area= area,
        grid= [0.1, 0.1]
    )

    sp_area2 = mv.retrieve(
        date= adate,
        time= 0,
        step= step,
        type= "fc",
        class_= "rd",
        levtype= "sfc",
        expver= expver2,
        param="sp",
        area= area,
        grid= [0.1, 0.1]
    )

    
    mv.write(skt_fc1,skt_area1)
    mv.write(skt_fc2,skt_area2)

    mv.write(t2_fc1,t2_area1)
    mv.write(t2_fc2,t2_area2)

    mv.write(sp_fc1,sp_area1)
    mv.write(sp_fc2,sp_area2)

## Read data

In [8]:
skt_1 = mv.read(skt_fc1) 
skt_2 = mv.read(skt_fc2) 
t2_1 = mv.read(t2_fc1) 
t2_2 = mv.read(t2_fc2)
sp_1 = mv.read(sp_fc1) 
sp_2 = mv.read(sp_fc2)

## Extract thermo profile for a given location

In [9]:
prof_fc1 = mv.thermo_grib(
    data=g1, coordinates=location, dew_point_formulation="saturation_over_water"
)

prof_fc2 = mv.thermo_grib(
    data=g2, coordinates=location, dew_point_formulation="saturation_over_water"
)

skt_point_oper = mv.nearest_gridpoint(skt_1, location)
skt_point_destine = mv.nearest_gridpoint(skt_2, location)
t2_point_oper = mv.nearest_gridpoint(t2_1, location)
t2_point_destine = mv.nearest_gridpoint(t2_2, location)
sp_point_oper = mv.nearest_gridpoint(sp_1, location)
sp_point_destine = mv.nearest_gridpoint(sp_2, location)

skt_point_oper = skt_point_oper-273.15
skt_point_destine = skt_point_destine-273.15
t2_point_oper = t2_point_oper-273.15
t2_point_destine = t2_point_destine-273.15
sp_point_oper = sp_point_oper/100
sp_point_destine = sp_point_destine/100


## Plotting configuration

In [10]:
# compute parcel path - maximum cape up to 700 hPa. 
if cape_data == "prof_fc1":
    parcel = mv.thermo_parcel_path(prof_fc1, mode="mucape", top_p=700)
else:
    parcel = mv.thermo_parcel_path(prof_fc2, mode="mucape", top_p=700)    

# create plot object for parcel areas
parcel_area = mv.thermo_parcel_area(parcel)

# create plot object for parcel path
parcel_vis = mv.xy_curve(parcel["t"], parcel["p"], "charcoal", "dash", 6)

# fc: define t and td profile style
prof_vis_fc1 = mv.mthermo(
    legend="on",
    legend_user_text="FC tco1279",
    thermo_temperature_line_colour="red",
    thermo_temperature_line_thickness=4,
    thermo_dewpoint_line_colour="red",
    thermo_dewpoint_line_thickness=4,
)

prof_vis_fc2 = mv.mthermo(
    legend="on",
    legend_user_text="FC tco2559",
    thermo_temperature_line_colour="blue",
    thermo_temperature_line_thickness=4,
    thermo_dewpoint_line_colour="blue",
    thermo_dewpoint_line_thickness=4,
)

# fc: define wind plotting style
prof_wind_style_fc1 = mv.mwind(
    wind_thinning_factor=2,
    wind_field_type="flags",
    wind_flag_colour="red",
    wind_flag_length=0.8,
    wind_flag_origin_marker="dot",
    wind_flag_origin_marker_size=0.2,
)

prof_wind_style_fc2 = mv.mwind(
    wind_thinning_factor=2,
    wind_field_type="flags",
    wind_flag_colour="blue",
    wind_flag_length=0.8,
    wind_flag_origin_marker="dot",
    wind_flag_origin_marker_size=0.2,
)

# define thermo grid
thermo_grid = mv.mthermogrid(
    thermo_isotherm_colour="RGB(0.2577,0.6364,0.5039)",
    thermo_isotherm_reference_colour="blue",
    thermo_isotherm_label_font_size=0.4,
    thermo_isobar_label_font_size=0.4,
    thermo_dry_adiabatic_colour="grey",
    thermo_dry_adiabatic_label_frequency=2,
    thermo_mixing_ratio_colour="RGB(0.2577,0.6364,0.5039)",
    thermo_mixing_ratio_label_colour="RGB(0.2577,0.6364,0.5039)",
    thermo_mixing_ratio_label_font_size=0.4,
)

# define the thermodynamic view (WINTER)
#view = mv.thermoview(
#    type=profile_type,
#    minimum_temperature=-80,
#    maximum_temperature=20,
#    top_pressure=300,
#    bottom_pressure= 1050,
#    thermo_grid=thermo_grid,
#    subpage_clipping="on",
#)

# define the thermodynamic view
view = mv.thermoview(
    type=profile_type,
    minimum_temperature=-90,
    maximum_temperature=0,
    top_pressure=300,
    bottom_pressure= 1050,
    thermo_grid=thermo_grid,
    subpage_clipping="on",
)

title_txt = build_title_text(prof_fc1)
    
title = mv.mtext(text_lines=title_txt, text_font_size=0.7, text_colour="charcoal")

# define text lines for info box
txt = []
txt.append("     Mode: " + parcel["start"]["mode"])
txt.append("  Start p: {:.0f} hPa".format(parcel["start"]["p"]))
txt.append("  Start t: {:.1f} C".format(parcel["start"]["t"]))
txt.append(" Start td: {:.1f} C".format(parcel["start"]["td"]))
txt.append("     CAPE: {:.3f} J/kg".format(parcel["cape"]))
txt.append("      CIN: {:.3f} J/kg".format(parcel["cin"]))
txt.append("       LI: {:.1f} K".format(parcel["li"]))

if parcel["lcl"] is not None:
    txt.append("    LCL p: {:.0f} hPa".format(parcel["lcl"]["p"]))
    txt.append("    LCL t: {:.1f} C".format(parcel["lcl"]["t"]))

if parcel["lfc"] is not None:
    txt.append("    LFC p: {:.0f} hPa".format(parcel["lfc"]["p"]))
    txt.append("    LFC t: {:.1f} C".format(parcel["lfc"]["t"]))

if parcel["el"] is not None:
    txt.append("     EL p: {:.0f} hPa".format(parcel["el"]["p"]))
    txt.append("     EL t: {:.1f} C".format(parcel["el"]["t"]))


# create info box - ensure font is monospace
info_box = mv.mtext(
    text_lines=txt,
    text_font="courier",
    text_font_size=0.5,
    text_colour="black",
    text_justification="left",
    text_mode="positional",
    text_box_x_position=4,
    text_box_y_position=5.4,
    text_box_x_length=6,
    text_box_y_length=len(txt) * 0.6 + 0.4,
    text_box_blanking="on",
    text_border="on",
    text_border_colour="black",
)

x_vis_oper = mv.input_visualiser(
    input_x_values = skt_point_oper,
    input_y_values = sp_point_oper,
    input_values   = 10
    )
 
x_sym_oper = mv.msymb(   
    legend="on",
    legend_user_text="skt tco1279",   
    symbol_type           = "text",
    symbol_colour         = "red",
    symbol_text_list      = "o",
    symbol_text_font_size = 0.5,
    symbol_text_font_style = "bold"
    )

x_vis_destine = mv.input_visualiser(
    input_x_values = skt_point_destine,
    input_y_values = sp_point_destine,
    input_values   = 10
    )
 
x_sym_destine = mv.msymb(
    legend="on",
    legend_user_text="skt tco2559",
    symbol_type           = "text",
    symbol_colour         = "blue",
    symbol_text_list      = "o",
    symbol_text_font_size = 0.5,
    symbol_text_font_style = "bold"
    )

x_vis_oper_2t = mv.input_visualiser(
    input_x_values = t2_point_oper,
    input_y_values = sp_point_oper,
    input_values   = 10
    )
 
x_sym_oper_2t = mv.msymb(
    legend="on",
    legend_user_text="2t tco1279",
    symbol_type           = "text",
    symbol_colour         = "red",
    symbol_text_list      = "x",
    symbol_text_font_size = 0.5,
    symbol_text_font_style = "bold"
    )

x_vis_destine_2t = mv.input_visualiser(
    input_x_values = t2_point_destine,
    input_y_values = sp_point_destine,
    input_values   = 10
    )
 
x_sym_destine_2t = mv.msymb(
    legend="on",
    legend_user_text="2t tco2559",
    symbol_type           = "text",
    symbol_colour         = "blue",
    symbol_text_list      = "x",
    symbol_text_font_size = 0.5,
    symbol_text_font_style = "bold"
    )

# define positional legend
legend = mv.mlegend(
    legend_text_colour="black",
    legend_box_mode="positional",
    legend_display_type="disjoint",
    legend_text_font_size=0.5,
    legend_entry_plot_direction="column",
    legend_box_x_position=26,
    legend_box_y_position=5,
    legend_box_x_length=1,
    legend_box_y_length=7,
    legend_entry_text_width=5,
)

print(skt_point_oper)
print(skt_point_destine)
print(t2_point_oper)
print(t2_point_destine)
print(sp_point_oper)
print(sp_point_destine)



-6.760702514648415
-6.428076171874977
-6.603964233398415
-6.261358642578102
1009.600625
1008.46125


## Plotting

In [13]:
# define the output plot file
#mv.setoutput(mv.pdf_output(output_name="tephigram_fc"))

if cape_instability:
    mv.plot(
        view,
        prof_fc1,
        prof_vis_fc1,
        prof_wind_style_fc1,
        prof_fc2,
        prof_vis_fc2,
        prof_wind_style_fc2,
        x_vis_oper,
        x_sym_oper,
        x_vis_destine,
        x_sym_destine,
        x_vis_oper_2t,
        x_sym_oper_2t,
        x_vis_destine_2t,
        x_sym_destine_2t,
        title,
        info_box,
        legend,
    )
            
else:        
    mv.plot(
        view, 
        prof_fc1,
        prof_vis_fc1,
        prof_wind_style_fc1,
        prof_fc2,
        prof_vis_fc2,
        prof_wind_style_fc2,
        x_vis_oper,
        x_sym_oper,
        x_vis_destine,
        x_sym_destine,
        x_vis_oper_2t,
        x_sym_oper_2t,
        x_vis_destine_2t,
        x_sym_destine_2t,
        title,
        legend,    
    )      

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')